In [6]:
!pip install -q langchain langchain_community openai python-dotenv cassio datasets tiktoken PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [5]:
from langchain.vectorstores import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

import cassio

In [7]:
from PyPDF2 import PdfReader

In [11]:
from google.colab import userdata

ASTRADB_APPLICATION_TOKEN=userdata.get('ASTRADB_APPLICATION_TOKEN')
ASTRADB_ID=userdata.get('ASTRADB_ID')

OPENAI_API_KEY=userdata.get('OPEN_AI_KEY')

In [46]:
pdfreader = PdfReader('./attention.pdf')

In [ ]:
raw_text = ''
for i,page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

raw_text

In [20]:
cassio.init(token=ASTRADB_APPLICATION_TOKEN, database_id=ASTRADB_ID)

In [42]:
llm=OpenAI(openai_api_key=OPENAI_API_KEY,)
embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model='text-embedding-3-small')

In [26]:
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None
)


In [28]:
from langchain.text_splitter import CharacterTextSplitter

texts = CharacterTextSplitter(
    separator='\n',
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
).split_text(raw_text)

In [37]:
len(texts), texts[0]

(102,
 'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto')

In [38]:
astra_vector_store.add_texts(texts[:25])
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [44]:
while True:

  query_text = input("Enter your question or type quit to exit: ")

  if query_text == 'quit':
    break
  elif query_text == '':
    continue

  print("\n\nQuestion: ", query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("\nAnswer: ", answer)

  print("\nFirst document by relavance: ")

  for doc , score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print("[%0.4f] \"%s ... \"" %(score, doc.page_content[:84]))



Enter your question or type quit to exit: What is attention ?


Question:  What is attention ?



Answer:  Attention is a mechanism used in sequence modeling and transduction models that allows for modeling of dependencies without regard to their distance in the input or output sequences. It involves using queries and keys to compute the weights on values, and can be used in tasks such as reading comprehension, summarization, and textual entailment.

First document by relavance: 


[0.7574] "described in section 3.2.
Self-attention, sometimes called intra-attention is an att ... "
[0.7096] "3Scaled Dot-Product Attention
 Multi-Head Attention
Figure 2: (left) Scaled Dot-Prod ... "
[0.7031] "Attention mechanisms have become an integral part of compelling sequence modeling an ... "
[0.6993] "We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The inpu ... "
Enter your question or type quit to exit: quit
